In [1]:
import numpy as np
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cis-ieee-unb-renting-houce-prices/train.csv
/kaggle/input/cis-ieee-unb-renting-houce-prices/test.csv


### Load Data set
Primeira etapa, separar os dados de entradas e de saída de ambos os data_set

In [2]:
train_set = pd.read_csv("/kaggle/input/cis-ieee-unb-renting-houce-prices/train.csv")
test_set = pd.read_csv("/kaggle/input/cis-ieee-unb-renting-houce-prices/test.csv")

/tmp/ipykernel_33/2673490125.py:1: DtypeWarning: Columns (0,21,24,29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_set = pd.read_csv("/kaggle/input/cis-ieee-unb-renting-houce-prices/train.csv")


#### Análise de elementos faltantes
O primeiro passo será a contagem de elementos faltantes em cada coluna. Depois, iremos dividir pela quantidade total de elementos e multiplicar por 100 para descobrir sua porcentagem *porcentagem*

In [3]:
train_set.isnull().sum() / train_set.shape[0] * 100

id                                               0.000000
listing_url                                      0.000000
scrape_id                                        0.000170
last_scraped                                     0.000000
name                                             0.190447
summary                                          5.041737
space                                           39.495928
description                                      2.299644
experiences_offered                              0.000000
neighborhood_overview                           46.108340
minimum_minimum_nights                          23.290613
maximum_minimum_nights                          23.290613
minimum_maximum_nights                          23.290613
maximum_maximum_nights                          23.290613
minimum_nights_avg_ntm                          23.290613
maximum_nights_avg_ntm                          23.290613
number_of_reviews_ltm                           23.290613
calculated_hos

In [4]:
test_set.isnull().sum() / test_set.shape[0] * 100

id                                               0.000000
listing_url                                      0.000000
scrape_id                                        0.000000
last_scraped                                     0.000000
name                                             0.165281
summary                                          4.946182
space                                           39.408764
description                                      2.201194
experiences_offered                              0.000000
neighborhood_overview                           45.966944
minimum_minimum_nights                          23.543845
maximum_minimum_nights                          23.543845
minimum_maximum_nights                          23.543845
maximum_maximum_nights                          23.543845
minimum_nights_avg_ntm                          23.543845
maximum_nights_avg_ntm                          23.543845
number_of_reviews_ltm                           23.543845
calculated_hos

#### Lidando com elementos faltantes
Iremos eliminar todas as colunas com mais de 20% de elementos faltantes, considerando que a maioria das colunas que possui elementos faltantes possui uma porcentagem inferior a 1%.

In [5]:
columns_to_drop = ["space", "neighborhood_overview", "minimum_minimum_nights",
"maximum_minimum_nights",
"minimum_maximum_nights",
"maximum_maximum_nights",
"minimum_nights_avg_ntm",
"maximum_nights_avg_ntm",
"number_of_reviews_ltm",
"calculated_host_listings_count_entire_homes",
"calculated_host_listings_count_private_rooms",
"calculated_host_listings_count_shared_rooms",]

train_set = train_set.drop(columns_to_drop,axis =1)
test_set = test_set.drop(columns_to_drop,axis =1)

Além do drop das colunas com elementos faltantes, também iremos eliminar as colunas cujas valores não podem ser quantificados, ou seja, colunas no qual a maioria terá valores "únicos" e que dificilmente poderiam ser avaliados pelo modelo. 
"last_scraped" será removido pois last_scraped_id é sua representação numérica

In [6]:
columns_to_drop = ["name", "summary", "description", "amenities", "listing_url", "experiences_offered", "id", "last_scraped"]
test_ids = test_set.get["id"]


train_set = train_set.drop(columns_to_drop,axis =1)
test_set = test_set.drop(columns_to_drop,axis =1)

TypeError: 'method' object is not subscriptable

In [ ]:
train_set.columns

Após o drop das colunas acima, as demais restantes possuem poucos elementos faltantes. No entanto, esses elementos são tanto do tipo Numerico, tanto categórico. Além disso, nota-se que algumas dessas colunas como número de banheiros e se o host é super host estão sim relacionados com outras colunas do data_set. Faz sentido pensarmos que o número de banheiros em uma casa está relacionado com o número de quartos, ou que o número de camas está relacionado com o número de banheiros.

Por isso, a abordagem escolhida para lidar com os elementos faltantes será *Multiple Imputation*

No entanto, devido ao fato de que o import da biblioteca não funcionou, estaremos utilizando a moda para substituir o valor!

In [ ]:
train_mode_value = train_set.mode()
test_mode_value = test_set.mode()
train_set = train_set.fillna(train_mode_value.iloc[0])
test_set = test_set.fillna(test_mode_value.iloc[0])


In [ ]:
print("train_set property_type: ", train_set["property_type"].unique())
print("test_set property_type: ", test_set["property_type"].unique())

print("train_set room_type: ", train_set["room_type"].unique())
print("test_set room_type: ", test_set["room_type"].unique())

print("train_set cancellation_policy:", train_set["cancellation_policy"].unique())
print("test_set cancellation_policy:", test_set["cancellation_policy"].unique())


print("train_set extra_people:", train_set["extra_people"].unique())
print("test_set extra_people:", test_set["extra_people"].unique())



### Data encoding
Cada coluna com dados do tipo *categorical* serão codificados segundo suas especificações:
- property_type: Label Encoding
- room_type: Ordinal Encoding
- cancellation_policy: Ordinal Encoding
- extra_people: Function


One-Hot-Encoding não foi aplicado devido ao fato do grande consumo de memória e ao grande número de valores únicos no banco de dados de treinamento. 
Para room_type e cancellation_policy foi aplicaddo o Ordinal Encoding baseado no fato de que existe uma hierarquia em suas propriedades enquanto que para extra_people, como todos os valores correspondem a dollar, iremos aplicar uma função que transforma esse valores para inteiro

Para as colunas booleanas precisamos alterar os valores no banco de dados para conseguir treinar

In [ ]:
def dollar_to_float(dollar):
    if(not "$" in dollar):
        return 0.0
    return float(dollar.replace('$',"").replace(",",""))

In [ ]:
def boolean_to_value(boolean):
    if(boolean == "t" or boolean == "T"):
        return 1
    else:
        return 0

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
label_encoder = LabelEncoder()

train_set["property_type"] = label_encoder.fit_transform(train_set["property_type"])
test_set["property_type"] = label_encoder.fit_transform(test_set["property_type"])

In [ ]:
room_type_hierarchy = [ '2019-03-06','Shared room', 'Private room', 'Hotel room', 'Entire home/apt']
cancellation_hierarchy = ['flexible', 'moderate', 'strict','strict_14_with_grace_period','super_strict_30', 'super_strict_60']

ordinal_encoder_room = OrdinalEncoder(categories=[room_type_hierarchy])
ordinal_encoder_cancellation = OrdinalEncoder(categories=[cancellation_hierarchy])

train_set['room_type'] = ordinal_encoder_room.fit_transform(train_set[['room_type']])
test_set['room_type'] = ordinal_encoder_room.fit_transform(test_set[['room_type']])

train_set['cancellation_policy'] = ordinal_encoder_cancellation.fit_transform(train_set[['cancellation_policy']])
test_set['cancellation_policy'] = ordinal_encoder_cancellation.fit_transform(test_set[['cancellation_policy']])


In [ ]:
train_set["extra_people"] = train_set["extra_people"].apply(dollar_to_float)
test_set["extra_people"] = test_set["extra_people"].apply(dollar_to_float)

In [ ]:
train_set["host_is_superhost"] = train_set["host_is_superhost"].apply(boolean_to_value)
test_set["host_is_superhost"] = test_set["host_is_superhost"].apply(boolean_to_value)

train_set["instant_bookable"] = train_set["instant_bookable"].apply(boolean_to_value)
test_set["instant_bookable"] = test_set["instant_bookable"].apply(boolean_to_value)

In [ ]:
train_set["price"] = train_set["price"].apply(dollar_to_float)

#### Eliminando objetos com valores errados
Com a limpeza do data_set, agora iremos eliminar os objetos de teste e treinamento que não sejam do tipo numérico.

In [ ]:
train_set.dtypes

In [ ]:
test_set.dtypes

In [ ]:
def is_valid_value(value):
    if(type(value) is int or type(value) is float):
        return True
    if(type(value) is str):
        if(value.replace(".","").isnumeric()):
            return True
    return False

Com base nos tipos, precisaremos aplicar apenas no data_set de treinamento


In [ ]:
train_set = train_set[train_set["host_listings_count"].apply(is_valid_value)]
train_set = train_set[train_set["accommodates"].apply(is_valid_value)]

### Separando dataset em entradas e saídas

In [ ]:
train_set_x = train_set.drop('price', axis = 1)
train_set_y = train_set['price']

test_set_x = test_set

### Normalizando data_set

In [ ]:
train_set_x

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
columns_to_scale = ["extra_people", "number_of_reviews", "scrape_id", "latitude", "longitude"]

train_set_x[columns_to_scale] = scaler.fit_transform(train_set_x[columns_to_scale])
train_set_x


In [ ]:
test_set_x[columns_to_scale] = scaler.transform(test_set_x[columns_to_scale])
test_set_x

### Redução de dimensionalidade
Existem diversas táticas de redução de dimensionalidade que podem ser atribuídos no data_set, seu objetivo é evitar overfitting de treinamento e diminuir a complexidade da previsão, tendo em vista a grande quantidade de variáveis. A técnica utilzada será a *PCA*.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
train_pca_x = PCA(n_components=5, whiten=True)
train_pca_x = train_pca_x.fit(train_set_x)
train_pca_x = train_pca_x.transform(train_set_x)
train_pca_x


In [ ]:
test_pca_x = PCA(n_components=5, whiten=True)
test_pca_x = test_pca_x.fit(test_set_x)
test_pca_x = test_pca_x.transform(test_set_x)
test_pca_x


### Modelos de treinamento

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression

#### Linear Regression

In [ ]:
linear_reg_model = LinearRegression()
linear_reg_model.fit(train_pca_x, train_set_y)

In [ ]:
linear_train_predic = linear_reg_model.predict(train_pca_x)
linear_train_predic

In [ ]:
train_set_y

#### Logistic Regression

In [ ]:
ridge_reg_model = Ridge()
ridge_reg_model.fit(train_pca_x, train_set_y)

In [ ]:
ridge_predic = ridge_reg_model.predict(train_pca_x)
ridge_predic

### Analise de erros

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
r2_linear = r2_score(train_set_y, linear_train_predic)
print("r2 score linear: ", r2_linear)

mse_linear = mean_squared_error(train_set_y, linear_train_predic)
print("mse error linear: ", mse_linear)

mae_linear = mean_absolute_error(train_set_y, linear_train_predic)
print("mae error linear: ", mae_linear)


In [ ]:
r2_ridge = r2_score(train_set_y, linear_train_predic)
print("r2 score ridge: ", r2_linear)

mse_ridge = mean_squared_error(train_set_y, linear_train_predic)
print("mse error ridge: ", mse_linear)

mae_linear = mean_absolute_error(train_set_y, linear_train_predic)
print("mae error ridge: ", mae_linear)


In [ ]:
ridge_prediction_test = ridge_reg_model.predict(test_pca_x)
linear_prediction_test = linear_reg_model.predict(test_pca_x)

In [ ]:
sub = pd.DataFrame({
        "id": test_ids,
        "ridge_prediction": ridge_prediction_test,
        "linearPrediction": linear_prediction_test
    })


sub.to_csv("submission.csv")